In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
icd = pd.read_csv('../webapp/database/omop_vocabulary/icd/CONCEPT.csv',
                  sep='\t', usecols=['concept_id', 'concept_code', 'vocabulary_id'])
icd_relationship = pd.read_csv('../webapp/database/omop_vocabulary/icd/CONCEPT_RELATIONSHIP.csv',
                               sep='\t', usecols=['concept_id_1', 'concept_id_2', 'relationship_id'])
icd_relationship = icd_relationship[icd_relationship['relationship_id']=='Subsumes']

commorbidities_raw = pd.read_excel("commorbidities.xlsx")
commorbidities_raw = commorbidities_raw.rename(columns={
    'Commorbidities':'commorbidities',
    'Elixhauser’s original ICD-9-CM':'icd9_1',
    'Elixhauser AHRQWeb ICD-9-CM':'icd9_2',
    'Enhanced ICD-9-CM':'icd9_3',
    'ICD-10':'icd10'
})

/Users/anthonydubois/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def get_concept_subsumes(concept_code, vocabulary_id, relationship='itself'):
    t = icd[(icd['concept_code']==concept_code) & (icd['vocabulary_id']==vocabulary_id)]

    if len(t) > 0:
        concept_id = t.iloc[0]['concept_id']
        subsumes = icd_relationship[icd_relationship['concept_id_1']==concept_id].concept_id_2.tolist()
        return [concept_id] + subsumes
    return []


In [4]:
com = pd.DataFrame()

for i in range(len(commorbidities_raw)):
    l1 = []
    l2 = []
    for col in ['icd9_1', 'icd9_2', 'icd9_3']:
        raw = str(commorbidities_raw[col][i])
        if raw == 'nan':
            continue
        raw = raw.replace('\n', '') # Remove some characters
        raw = raw.replace(' ', '') # Remove space
        l1 = l1 + raw.split(',')
        l1 = list(dict.fromkeys(l1)) # Remove duplicates
        
    raw = str(commorbidities_raw['icd10'][i])
    if raw == 'nan':
        continue
    raw = raw.replace('\n', '') # Remove some characters
    raw = raw.replace(' ', '') # Remove space
    l2 = raw.split(',')
    l2 = list(dict.fromkeys(l2)) # Remove duplicates
        
    df = pd.DataFrame({
        'COMMORBIDITIES':[commorbidities_raw['commorbidities'][i]],
        'ICD9CM':[l1],
        'ICD10':[l2],
        'CONCEPT_ID':[[np.nan]]
    })
    com = com.append(df, ignore_index=True)
    
for i in range(len(com)):
    l = []
    for vocabulary in ['ICD9CM', 'ICD10']:
        for element in com[vocabulary][i]:
            if '-' in element:
                p = element.replace('x', '')
                a = p.split('-')
                if a[0][0].isalpha():
                    q = a[0][0]
                else:
                    q = ''
                b = [re.sub(r'[A-Z]+', '', s) for s in a]
                c = round(float(b[1]) - float(b[0]), 6)*100
                if c %10 == 0:
                    c = c/10
                c = round(c, 6)
                d = [q + str(round(i, 6)) + 'x' for i in np.linspace(float(b[0]), float(b[1]), c+1)]
                for a in d:
                    z = a.replace('.x', '')
                    z = z.replace('x', '')
                    l = l + get_concept_subsumes(z, vocabulary)
                continue
            
            elif 'x' in element:
                a = element.replace('.x', '')
                a = a.replace('x', '')
                l = l + get_concept_subsumes(a, vocabulary)
                
            else:
                l = l + get_concept_subsumes(element, vocabulary)
                
    l = list(dict.fromkeys(l)) # Remove duplicates
    com.at[i, 'CONCEPT_ID'] = l

In [7]:
com[['COMMORBIDITIES', 'CONCEPT_ID']].to_csv('commorbidities_magic.csv', index=False)

In [5]:
for index, row in com.iterrows():
    print(row['COMMORBIDITIES'])
    print(len(row['CONCEPT_ID']))

Congestive heart failure
55
Cardiac arrhythmias
72
Valvular disease
68
Pulmonary circulation Disorders
27
Peripheral vascular disorders
88
Hypertension,uncomplicated
11
Hypertension,complicated
78
Paralysis
86
Other neurological disorders
143
Chronic pulmonary disease
82
Diabetes,uncomplicated
41
Diabetes,complicated
66
Hypothyroidism
12
Renal failure
47
Liver disease
79
Peptic ulcer disease excluding bleeding
45
Lymphoma
328
Metastatic cancer
58
Solid tumor without metastasis
602
Rheumatoid arthritis/collagen vascular diseases
376
Coagulopathy
30
Obesity
11
Weight loss
9
Fluid and electrolyte disorders
29
Blood loss anemia
8
Deficiency anemia
29
Drug abuse
187
Psychoses
185
Depression
67
Alcohol abuse
72
AIDS/H1V
19
